# Čtení QR kódů řádkovou kamerou
Cvičení je zaměřené na práci s řádkovými kamerami, filtry a barevným osvětlením. Řádkové kamery jsou díky svým specifickým vlastnostem vynikající pro řadu úloh, typickým použitím je snímání nekonečného pásu ve výrobních linkách. Poněvadž nekonečné pásy nemáme v dostatečném počtu, budeme je simulovat pomocí povrchu otočného válce - plechovky. Na povrchu plechovky bude nalepen QR kód, který chceme přečíst.

Protože řádkové kamery, které máme k dispozici, umí obraz snímat pouze ve stupních šedé, bude v případě snímání barevného objektu nutno zajistit dostatečný kontrast mezi původními barevnými odstíny. K tomu budou k dispozici různá barevná světla a filtry, jejichž vhodnou kombinaci bude třeba zvolit k dosažení optimálních výsledků.

## Řádkové kamery
Řádková kamera neboli řádkový skener je speciální druh kamery, která snímá pouze 1 řádek. Díky tomu je schopna dosáhnout obrovských frekvencí snímání (až ticíce řádků za 1 sekundu). V labu máme kamery [Basler Racer](https://www.baslerweb.com/en/products/cameras/line-scan-cameras/racer/) o velikosti řádku 6k, 8k a 12k pixelů.

<img src="images/racer.png" width="25%">

Z podstaty skeneru však vyplývá, že se objekt pod kamerou musí pohybovat. V případě, že se objekt nebude pohybovat, vytvoří kamera obrázek, který bude mít všechny řádky totožné (pouze ten 1, který kamera snímá). 

Tím, že je kamera schopná dosahovat neskutečných frekvencí snímání dále umožňuje získávat obrazová data v neskutečně velkém rozlišení. Na druhou stranu nás tím nutí nastavovat nízkou dobu expozice a tedy je zapotřebí mnohem více světla.

Důležité pojmy související s řádkovými kamerami:
- **frame** - během komunikace s počítačem řádková kamera typicky neposílá každý řádek zvlášť, ale ukládá si je do lokálního bufferu a odesílá je pohromadě v jednom "frame" (rámci). Pylon Viewer pak řádky z jednoho frame zobrazuje v jednom obrázku, nad sebou. Počet řádků je možné v konfiguraci kamery nastavit parametrem *height*.
- **acquisition** - režim snímání kamery, typicky single shot/continuous. Single shot vyfotí pouze jeden frame, režim continuous rámce snímá nepřetržitě. V aplikacích jako jsou nekonečné pásy je možné použít buď single shot akvizici a mít zvláštní trigger pro každý objekt, nebo continuous akvizici a objekty rozlišovat v post-processingu.

---

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:
- [`connect_camera(...)`](../svz.ipynb#connect_functions)


- [`show_images(...)`](../svz.ipynb#show_functions)
- [`plot_images(...)`](../svz.ipynb#show_functions)


- [`to_gray(...)`](../svz.ipynb#preprocessing_functions)
- [`to_hsv(...)`](../svz.ipynb#preprocessing_functions)
- [`negative(...)`](../svz.ipynb#preprocessing_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)
- [`rotate(...)`](../svz.ipynb#cart_polar_functions)
- [`crop_by_bounding_rect(...)`](../svz.ipynb#preprocessing_functions)
- [`crop_contour(...)`](../svz.ipynb#preprocessing_functions)
- [`contour_to_image(...)`](../svz.ipynb#preprocessing_functions)
- [`resize(...)`](../svz.ipynb#preprocessing_functions)


- [`filtration_median(...)`](../svz.ipynb#filtration_functions)
- [`fill_holes(...)`](../svz.ipynb#filtration_functions)


- [`to_intensity(...)`](../svz.ipynb#others)
- [`logical_and(...)`](../svz.ipynb#others)

---

### Úkol 1
Seznamte se s řádkovou kamerou a jejími parametry, jejichž nastavení je pro správné snímání kritické. Kameru zaostřete na mřížkový podklad stojanu a celou zaostřenou kameru poté pomocí kličky zdvihněte, aby byla kamera zaostřená na vršek plechovky. Nakonec získejte snímek QR kódu, který přečtěte a vizualizujte na původním snímku.

<div style="color: blue; text-align: right">[ 1 bod ]</div>

#### 1) Získejte snímek černobílého QR kódu
Za stálého otáčení plechovky zachyťte snímek černobílého QR kódu na jejím povrchu. Rychlost otáčení plechovky je třeba udržovat pokud možno konstantní, aby různé části snímku QR kódu nebyly různě protáhlé - knihovna, kterou používáme, by pak měla problém kód přečíst. Důležité také je, aby vyfocený QR kód byl co nejpodobnější obdélníku, tedy aby nebyl zkosený či jinak deformovaný.

In [ ]:
image = ...(...) ###
show_images(image)

#### 5) Předzpracujte snímek QR kódu
V této sekci je nutné zejména přeškálovat obrázek tak, aby nebyl nevhodně protáhlý z důvodu nekorespondující snímkové frekvence vůči rychlosti otáčení plechovky. QR kód ve zpracovaném obrázku by měl být přibližně čtvercový (knihovna pyzbar, kterou použijeme ke čtení QR kódu, naštěstí poskytuje poměrně velkou toleranci). Obrázek můžete dále zpracovat dle libosti, například otočit nebo oříznout.

In [ ]:
height, width, channels = image.shape
width, height

In [ ]:
new_width, new_height = ... ###
resized_image = ... ###
show_images(resized_image)

#### 6) Pomocí knihovy pyzbar přečtěte z QR kódu zakódovaný text a získejte souřadnice bounding boxu
Pro čtení QR kódu z obrázku používáme funkci `decode` knihovny pyzbar. Funkce vrací informace o všech detekovaných QR kódech - především jejich dekódovaná data a souřadnice v původním obrázku. Pokud funkce QR kód ve vašem obrázku nedetekuje, zkuste data nasnímat znovu a případně kontaktujte cvičící, kteří vám poradí, co by bylo vhodné ve snímku zlepšit.

In [ ]:
import pyzbar.pyzbar as pyzbar

In [ ]:
decoded = pyzbar.decode(resized_image)
if len(decoded) < 1:
    raise RuntimeError("Nebyl nalezen zadny QR kod")
qr = decoded[0]
qr_data, qr_rect = qr.data, qr.rect
qr_data, qr_rect

Dekódovaná data vrací funkce ve formátu *bytes*, pro jejich převedení do řetězce je třeba provést další dekódování.

In [ ]:
qr_text = qr_data.decode("utf-8")
qr_text

#### 7) Do obrázku vykreslete ohraničující obdélník a vypište získaný text
Třešničkou na dortu je pro nás vizualizace QR kódu v obrázku - tj. zobrazení bounding boxu a vypsání dekódovaného textu vedle něj. K tomu je možné použít například funkce [cv2.rectangle()](https://docs.opencv.org/master/d6/d6e/group__imgproc__draw.html#ga07d2f74cadcf8e305e810ce8eed13bc9) a [cv2.putText()](https://docs.opencv.org/master/d6/d6e/group__imgproc__draw.html#ga5126f47f883d730f633d74f07456c576) knihovny opencv.

In [ ]:
qr_rect.left, qr_rect.top, qr_rect.width, qr_rect.height

In [ ]:
annotated_image = resized_image.copy()
cv2.rectangle(..., ..., ..., (0, 255, 0), 3) ###
cv2.putText(..., ..., ..., 0, 0.7, (0, 255, 0), 2) ###
show_images(annotated_image)

---

### Úkol 2
Druhý úkol je podobný prvnímu, opět v něm jde o nasnímání QR kódu a jeho dekódování. Zde je nicméně třeba vyfotit QR kód barevný - což je s černobílou kamerou problematické, jelikož výsledný černobílý snímek QR kódu nebude dostatečně kontrastní, aby jej knihovna pyzbar mohla přečíst. Je proto nutné během snímání zkoušet různé kombinace barevných světel a filtrů. Hodit se budou znalosti z přednášky o osvětlení - zkuste si vzpomenout, co jsou a jak se dají využít komplementární barvy.

<div style="color: blue; text-align: right">[ 1 bod ]</div>

#### 1) Získejte obrázek barevného QR kódu

In [ ]:
image = ...(...) ###
show_images(image)

#### 2) Upravte obrázek do podoby, v jaké půjde QR kód přečíst
Předzpracování se podobně jako v prvním úkolu skládá zejména z přeškálování obrázku, ale je potřeba provést další krok - QR kód ve vstupním obrázku nyní může mít invertované barvy, tedy světlý "potisk" na tmavém "pozadí". Knihovna pyzbar takový kód neumí přečíst, proto se vám může hodit funkce `negative()`.

In [ ]:
new_width, new_height = ... ###
resized_image = ... ###
show_images(resized_image)

In [ ]:
negative_image = ... ###
show_images(negative_image)

#### 3) Pomocí knihovy pyzbar přečtěte z QR kódu zakódovaný text

In [ ]:
decoded = pyzbar.decode(resized_image)
if len(decoded) < 1:
    raise RuntimeError("Nebyl nalezen zadny QR kod")
qr = decoded[0]
qr_data, qr_rect = qr.data, qr.rect
qr_data, qr_rect

In [ ]:
qr_text = qr_data.decode("utf-8")
qr_text